# **Zero-Shot Model**

In [ ]:
from transformers import pipeline

checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")

In [ ]:
from datasets import load_dataset
import kagglehub

# Datensatz von Kaggle herunterladen
path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")
print("Path to dataset files:", path)

# Datensatz als HF-Dataset laden
dataset = load_dataset("imagefolder", data_dir=path)
print(dataset)

In [ ]:
dataset['train'][0]['image']

In [4]:
# Liste aller Klassennamen (90 Tierarten)
label_names = dataset['train'].features['label'].names

In [5]:
from PIL import Image
from tqdm import tqdm

# Alle Tiernamen als mögliche Text-Labels für Zero-Shot Prediction
labels_animals = label_names

# Vorbereitung: Listen für echte und vorhergesagte Labels
true_labels = []
predicted_labels = []

In [ ]:
from tqdm import tqdm

# Anzahl zu evaluierender Bilder (kompletter Trainingsdatensatz)
num_samples = len(dataset['train'])

# Hauptloop: Zero-Shot Prediction für jedes Bild
for i in tqdm(range(num_samples)):

    # Hole PIL-Image aus Datensatz
    image = dataset['train'][i]['image']
    
    # Führe Zero-Shot Prediction durch mit CLIP
    results = detector(image, candidate_labels=labels_animals)

    # Sortiere nach höchster Score
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
    predicted_label = sorted_results[0]['label']

    # Wahres Label anhand der Label-ID
    true_label_idx = dataset['train'][i]['label']
    true_label = labels_animals[true_label_idx]
    
    # Ergebnisse speichern
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)


In [ ]:
# Evaluation der Vorhersagen mit klassischen Metriken
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Accuracy: Anteil korrekt vorhergesagter Labels
accuracy = accuracy_score(true_labels, predicted_labels)

# Precision: Verhältnis korrekt vorhergesagter Labels zu allen vorhergesagten Labels
precision = precision_score(true_labels, predicted_labels, average='weighted', labels=labels_animals, zero_division=0)

# Recall: Vollständigkeit der korrekt vorhergesagten Labels
recall = recall_score(true_labels, predicted_labels, average='weighted', labels=labels_animals, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
